In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

!unzip "/content/drive/My Drive/datasets/archive.zip" -d "/content"

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import os
import cv2

from tqdm import tqdm

In [ ]:
IMG_SIZE = 150

labels = ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"]

print('\nTrain Data')

data_train, y_detect_train, y_class_train = [], [], []

training_path = "/content/Training"

for label in labels:
  print(label)
  path = os.path.join(training_path, label)
  for img in tqdm(os.listdir(path)):
    image = cv2.imread(os.path.join(path, img))
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    data_train.append(image)
    y_class_train.append(label)
    tmp = 0 if label == "no_tumor" else 1
    y_detect_train.append(tmp)
X_train = np.array(data_train)

print('\nTest Data')

data_test, y_detect_test, y_class_test = [], [], []

testing_path = "/content/Testing"

for label in labels:
  print(label)
  path = os.path.join(testing_path, label)
  for img in tqdm(os.listdir(path)):
    image = cv2.imread(os.path.join(path, img))
    #print(img)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    data_test.append(image)
    y_class_test.append(label)
    tmp = 0 if label == 'no_tumor' else 1
    y_detect_test.append(tmp)
X_test = np.array(data_test)

In [ ]:
#X_train, y_class_train, y_detect_train = shuffle(X_train,y_class_train, y_detect_train, random_state=101)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()

y_class_train = le1.fit_transform(y_class_train)
y_class_test = le2.fit_transform(y_class_test)
y_detect_train = le3.fit_transform(y_detect_train)
y_detect_test = le4.fit_transform(y_detect_test)

In [ ]:
TRAINING_DIR = training_path
TEST_DIR = testing_path

seed = 10

im_shape = (250, 250)

BATCH_SIZE = 16

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.3)
val_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.3)

In [ ]:
data_generator = ImageDataGenerator(
        validation_split=0.3,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #color_mode=grayscale,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

val_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=im_shape, shuffle=True, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")
# Generator para parte validação
validation_generator = val_data_generator.flow_from_directory(TRAINING_DIR, target_size=im_shape, shuffle=False, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")

# Generator para dataset de teste
test_generator = ImageDataGenerator(rescale=1./255)
test_generator = test_generator.flow_from_directory(TEST_DIR, target_size=im_shape, shuffle=False, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE)

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples
nb_test_samples = test_generator.samples
classes = list(train_generator.class_indices.keys())
print('Classes: '+str(classes))
num_classes  = len(classes)

In [ ]:
#base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(im_shape[0], im_shape[1], 3))

#base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(im_shape[0], im_shape[1], 3))

#base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(im_shape[0], im_shape[1], 3))

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(im_shape[0], im_shape[1], 3))

x = base_model.output
x = Flatten()(x)
x = Dense(300, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax', kernel_initializer='random_uniform')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freezing pretrained layers
for layer in base_model.layers:
    layer.trainable=True
    
optimizer = Adam()
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs = 50

# Saving the best model
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,verbose=1)
]

history = model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // BATCH_SIZE,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data=validation_generator,
        verbose = 1,
        validation_steps=nb_validation_samples // BATCH_SIZE)

Epoch 1/50
 82/125 [==================>...........] - ETA: 31:24 - loss: 1.9576 - accuracy: 0.2953

In [ ]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [ ]:
history = model.fit(X_train,y_class_train,validation_split=0.1, epochs =70, verbose=1, batch_size=32,
                   callbacks=[tensorboard,checkpoint,reduce_lr])

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()



best_accuracy=0.0

for epoch in range(20):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy